In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from le_kingmakers.preproc_text import process_tweets
from le_kingmakers.preproc_abbv import abbreviations
import regex
import string
import pickle
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from  sklearn.utils.random import sample_without_replacement
from sklearn.svm import SVC


### Import & Split Data

In [2]:
csv_path = os.path.join('/home/sbyhung/code/danm91/le_kingmakers/raw_data','training.1600000.processed.noemoticon.csv')
df = pd.read_csv(csv_path, header=None)
df = df.copy()

In [3]:
df.head(3)

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [4]:
list(df.columns)

[0, 1, 2, 3, 4, 5]

In [5]:
display(df.shape)
df.head(3)

(1600000, 6)

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [10]:
# get sample for testing
sample_size = int(df.shape[0] * 0.05)
data_sample = df.sample(sample_size, random_state=0)
data_sample.shape

(80000, 6)

In [11]:
X = data_sample.iloc[:, 5]
y = data_sample.iloc[:, 0]
# X = pd.DataFrame(df.iloc[:, 5])
# y = pd.DataFrame(df.iloc[:, 0])
display(X.shape)
display(y.shape)
display(X.head(2))
display(y.head(2))

(80000,)

(80000,)

557138    wants to compete! i want hard competition! i w...
349381    It seems we are stuck on the ground in Amarill...
Name: 5, dtype: object

557138    0
349381    0
Name: 0, dtype: int64

In [12]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)


In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=0, stratify=y_temp)


In [14]:
display(X_train.shape)
display(X_test.shape)
display(X_val.shape)
display(y_val.value_counts())
display(y_test.value_counts())
display(y_train.value_counts())
display(y.value_counts())

(48000,)

(16000,)

(16000,)

4    8002
0    7998
Name: 0, dtype: int64

4    8002
0    7998
Name: 0, dtype: int64

4    24007
0    23993
Name: 0, dtype: int64

4    40011
0    39989
Name: 0, dtype: int64

### Process Data : Tokensize & Processing

In [15]:
## tokenise words
# word_tokens = X_train.apply(word_tokenize)

In [16]:
# ## inspect tokenized & processed data
# display(word_tokens.head(3))
# display(type(word_tokens))

In [17]:
### bag of words
vectorizer = CountVectorizer()
X_processed = vectorizer.fit_transform(X_train)

In [18]:
len(vectorizer.get_feature_names())

57226

In [19]:
X_processed.shape

(48000, 57226)

In [20]:
type(X_processed)

scipy.sparse.csr.csr_matrix

In [21]:
vectorizer.transform(X_val).shape

(16000, 57226)

In [22]:
vectorizer.transform(X_test).shape

(16000, 57226)

#### test on train results

In [23]:
model = SVC()
model.fit(X_processed, y_train)
model.score(X_processed, y_train)

0.9320208333333333

#### test on val results

In [24]:
X_val_processed = vectorizer.transform(X_val)

In [25]:
model.score(X_val_processed, y_val)

0.7775625

In [23]:
display(X_val.shape)
display(X_val_processed.shape)
display(y_val.shape)
display(X_processed.shape)


(1600,)

(1600, 11098)

(1600,)

(4800, 11098)

In [26]:
X_test_processed = vectorizer.transform(X_test)

In [27]:
model.score(X_test_processed, y_test)

0.7778125

### fitting on pipeline